In [3]:
! pip install elasticsearch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 kB 3.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 3.4 MB/s eta 0:00:00


In [6]:
try:
  import os
  import elasticsearch
  from elasticsearch import Elasticsearch
  import numpy as np
  import pandas as pd
  import sys
  import json
  from ast import literal_eval
  from tqdm import tqdm
  import datetime
  from elasticsearch import helpers
  

except Exception as e:
  print(f"error: {e}")

In [20]:
%%bash
# If you get 1 root & 2 daemon process then Elasticsearch instance has started successfully
ps -ef | grep elasticsearch

user       23528   23525  0 22:00 ?        00:00:00 grep elasticsearch


In [22]:
es = Elasticsearch(hosts = [{"host":"localhost", "port":9200,'scheme':'http'}])
# Check if python is connected to elasticsearch
print(es.ping())

True


In [23]:
# Elasticsearch 인덱스 생성
index_name = 'txt_data_index'
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name)

In [26]:
! pip install sentence-transformers
! pip install langchain unstructured openai chromadb Cython tiktoken pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.8 MB/s eta 0:00:00a 0:00:01
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-n

In [27]:
from langchain.text_splitter import CharacterTextSplitter

In [33]:
! pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [57]:
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

#OAuth 2.0 클라이언트 ID에서 다운로드한 JSON 파일 경로
CLIENT_SECRET_FILE = '/home/user/Downloads/<path>API_NAME = 'drive'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/drive']

# 인증 흐름 설정
flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)

# 사용자 인증 및 토큰 획득
credentials = flow.run_local_server(port=0)

# API 빌드
service = build(API_NAME, API_VERSION, credentials=credentials)

# 특정 폴더 ID
folder_id = '<folderid>'
# 파일 목록 가져오기
results = service.files().list(
    q=f"'{folder_id}' in parents",
    fields="files(id, name)",
).execute()
files = results.get('files', [])

files

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1022088326864-snb0mvj91nec76u2r43bcgb22p848fi7.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A34375%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=pFNNiKLeG4VDpdJ3rxrzZYCp4cyWRK&access_type=offline


[{'id': '1381AEn7Jbjme1E_BLLHvsrWWdQey1MtO',
  'name': '2023-1학기_휴학_및_복학_신청_안내(1차).txt'},
 {'id': '1Y4ch8q8dy84PE8EYEoHnjPo8dpctzAUI',
  'name': '1._2022-2_외국어성적우수장학금(제출점수)_신청_안내.txt'},
 {'id': '1KQU1IwA83OmmF_TjwSUUv7yA2j3NAEHu',
  'name': '(붙임1)_2023학년도_후기_일반대학원_신입생_모집요강.txt'},
 {'id': '16tq3W9WAroPDXMOEyb5rmxAgNDhnBGJR',
  'name': '2023-2_교내_성적장학생_선발관련_외국어성적표_제출_안내(공고용).txt'},
 {'id': '1Yg7gXgvP1JXbBJ9nuU_GWepD3Td0F8_k',
  'name': '2023년도_여름_계절수업_운영_계획1(2023.4.).txt'},
 {'id': '1Qz479oTMB40zclSjdc0jA4xZNzqK1U7B',
  'name': '2022년_2학기_푸름관,_아름관A동_퇴사_안내문.txt'},
 {'id': '12KAjaxhspiK0F2lwGbHpFIJXUEG4b6oB',
  'name': '3._외국어성적우수장학금지급기준_및_장학생_선발절차.txt'},
 {'id': '1GHTXRHjE3WyWRLCf1j8Iqx8qBfYz_dLT',
  'name': '★_2023학년도_금오공과대학교_신입생_입학식_등_학사안내.txt'},
 {'id': '1xh3eyuoFFct8tjYB8GgfL_kFPzWjqYjX',
  'name': '(붙임1)_2023학년도_복수전공_이수신청_안내.txt'},
 {'id': '1TEPGsT__lEcogf88gaKVXc1NVtpHOnBD',
  'name': '2022년_2학기_오름관_퇴사_안내문.txt'},
 {'id': '1RfCV1nLZ3f2bv-mRTd4L2jhgHOfgU-60',
  'name': '1.(학생용)공결시스템_사

In [61]:
# text_splitter 초기화
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)

i = 1

for file in files:
    file_id = file['id']
    file_name = file['name']
    content = service.files().get_media(fileId=file_id).execute()
    data = content.decode('utf-8')

# # 각 텍스트 파일을 읽어와서 Elasticsearch에 색인화
# for i, file_name in enumerate(file_list):
#     file_path = os.path.join(folder_path, file_name)
#     with open(file_path, 'r', encoding='utf-8') as file:
#         data = file.read()

    # text_splitter를 사용하여 텍스트 분리
    chunks = text_splitter.split_text(data)

    # 데이터를 Elasticsearch에 색인화
    for j, chunk in enumerate(chunks):
        document = {
            'text': chunk.strip(),  # 각 줄의 텍스트를 'text' 필드에 저장
            'file_name': file_name  # 파일 이름도 저장 (옵션)
        }
        es.index(index=index_name, id=i * len(file_name) + j + 1, body=document)
    i += 1


In [16]:
# 색인화가 완료되었다면 몇 초 기다려서 Elasticsearch에 적용되도록 합시다.
import time
time.sleep(5)

In [62]:
# 특정 쿼리로 검색하기
search_query = '장학생 선발 절차는?' # 여기 입력
search_body = {
    'query': {
        'match': {
            'text': search_query
        }
    },
    'size': 5 # 최대 5개의 문서만 반환(토큰 4000제한 때문에)
}

In [63]:
# 검색 실행
results = es.search(index=index_name, body=search_body)

In [64]:
# 결과 출력
print(f"검색어 '{search_query}'에 대한 검색 결과:")
for hit in results['hits']['hits']:
    print(hit['_source']['text'])

# top_hit = results['hits']['hits'][0]['_source']['text']
# print(top_hit)

검색어 '장학생 선발 절차는?'에 대한 검색 결과:
捤獥    汤捯    氠瑢    
2023-2학기 교내 성적장학생 선발관련
외국어성적표 제출안내
2023학년도 2학기 교내 성적우수 및 성적오름장학생 선발을 위한 외국어성적표 제출방법을 아래와 같이 안내하오니 기한 내에 제출하시기 바랍니다.
 ▷ 제출 대상 : 현재 재학생으로 2023-1학기 성적 이수 예정자
               (단, 2023년 8월 졸업(수료)예정자, 수업연한초과자, 졸업유예자 제외)
 ▷ 외국어시험성적 적용기준 : 2022. 6. 1. ~ 2023. 5. 31.까지 취득한 성적(응시일 기준)
   - (학생)외국어 성적 입력 및 성적표 제출(업로드) 기간 : 2023. 6. 28.(수)까지
       ※ 기한 내 우리대학 ‘BISKIT’에 입력 및 제출되어야 함
   - 학부(과) 승인완료 : 2023. 7. 7.(금)까지
       ※ 기한 내 우리대학 ‘BISKIT’에 승인완료 되어야 함(미승인시 성적장학대상에서 제외 됨)
 ▷ 제출 방법 :  붙임 ‘외국어성적 입력 방법’ 참고
氠瑢    
 외국어성적기준 및 승인절차 
氠瑢    
장학명
직전학기 최초성적
등록학기 학년별
어학 성적(TOEIC기준)
금오
등록금 전액
 평점 4.0 이상 
2학년
500점 이상
3학년
550점 이상
우등
등록금 80%
 평점 3.8 이상 
4학년 이상
600점 이상
성적오름
등록금 20%
 직전 2개 학기 성적이 평점평균 3.0 이상  
금오, 우등과 동일
※ (공통기준) 이수학점 : 직전학기 15학점 이상[단, 직전학기가 4학년(건축학전공은 5학년)인 경우  8학점 이상]
※ 위 기준은 기본자격기준이며, 
   금오, 우등은 기준 충족자 중 학부(과)에서 장학생 추천 → 최종 선발
   성적오름은 상승편차가 큰 자 중 최종 선발
▼
氠瑢    
“학생” 외국어성적 등록 및 승인요청
汤捯    ▷ BISKIT-학습활동-자기주도활동-자기주도활동신청-어학-등록-입력 및 

#gpt 연결

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install --upgrade llmx

In [ ]:
!pip install cohere

In [ ]:
!pip install llmx==0.0.15a0 cohere==<버전>

In [ ]:
!pip install openai==0.28

In [48]:
import openai

openai.api_key = '<Your_OpenAI_APIkey>'

In [59]:
# GPT를 사용하여 쿼리를 생성하는 함수
def generate_query(prompt):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=500 
    )
    return response.choices[0].text.strip()

In [85]:
# 결과가 있을 경우 결과를 GPT에 입력하여 응답 생성
if results.get('hits', {}).get('total', {}).get('value', 0) != 0:
    # 개행 문자 제거 후 최대 길이까지 자르기
    formatted_texts = [hit['_source']['text'].replace('\n', ', ') for hit in results['hits']['hits']]
    summary_prompt = '\n'.join(formatted_texts)

    # 리스트를 문자열로 조합
    formatted_result = ' '.join(formatted_texts)
    # GPT를 사용하여 쿼리를 생성합니다
    gpt_summary_query = generate_query(f"{summary_prompt} 중에서 '{search_query}'에 대한 답을 찾아줘.")

    # GPT로 생성된 요약 출력
    print("GPT로 생성된 답변:", gpt_summary_query)
else:
    print(f"검색어 '{search_query}'에 대한 검색 결과가 없습니다.")

GPT로 생성된 답변: 장학생 선발 절차는 다음과 같다. 
1. 자격기준 및 지원 자격: 공인외국어성적, 직전학기 성적, 기본자격 등
2. 제출서류 제출: 공인외국어시험 성적증명서, 직전학기 성적증명서 등
3. 등록금 및 장학금 지급: 등록금 20% 지원 및 장학금 우수자 선발
4. 장학생 선발: 기준 충족자 중 학부(과)에서 장학생 추천 → 최종 선발


#스냅샷 생성(임시)

In [ ]:
from elasticsearch import Elasticsearch

# Elasticsearch 연결 설정
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

# 백업할 인덱스 이름과 스냅샷 이름 정의
index_name = 'your_index_name'
snapshot_name = 'snapshot_1'

# 스냅샷 생성
def create_snapshot(index, snapshot_name):
    es.snapshot.create_repository(repository='my_backup', body={'type': 'fs', 'settings': {'location': '/path/to/your/snapshot/directory'}})
    es.snapshot.create(repository='my_backup', snapshot=snapshot_name, body={'indices': index})

# 스냅샷 복원
def restore_snapshot(snapshot_name, index):
    es.indices.close(index=index)
    es.indices.clear_cache(index=index)
    es.snapshot.restore(repository='my_backup', snapshot=snapshot_name, body={'indices': index})
    es.indices.open(index=index)

# 스냅샷 생성
create_snapshot(index_name, snapshot_name)

# 스냅샷 복원 (예: 다른 인덱스로 복원)
restored_index_name = 'restored_index'
restore_snapshot(snapshot_name, restored_index_name)


#스냅샷 복원(임시)

일반적으로 Elasticsearch 스냅샷은 로컬 파일 시스템이나 원격 파일 시스템에 저장됩니다. 스냅샷 저장 위치는 Elasticsearch 설정 파일 (elasticsearch.yml)에서 지정하게 됩니다.

In [ ]:
from elasticsearch import Elasticsearch

# Elasticsearch 연결 설정
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

# 복원할 인덱스 이름 정의
restored_index_name = 'restored_index'

# 스냅샷 복원
def restore_snapshot(snapshot_name, index):
    es.indices.close(index=index)
    es.indices.clear_cache(index=index)
    es.snapshot.restore(repository='my_backup', snapshot=snapshot_name, body={'indices': index})
    es.indices.open(index=index)

# 스냅샷 복원
restore_snapshot('snapshot_1', restored_index_name)

# 복원된 인덱스에 대해 쿼리 수행
search_query = 'your_search_query'
search_body = {
    'query': {
        'match': {
            'text': search_query
        }
    }
}

# 쿼리 실행
results = es.search(index=restored_index_name, body=search_body)

# 결과 출력
print(f"검색어 '{search_query}'에 대한 검색 결과:")
for hit in results['hits']['hits']:
    print(f"파일: {hit['_source']['file_name']}, 텍스트: {hit['_source']['text']}")


#서버에서(임시)

Elasticsearch 서버 시작:
Elasticsearch 서버를 시작하여 새로운 설정이 적용되도록 합니다.

Linux에서는 다음과 같이 실행합니다:

bash
Copy code
sudo service elasticsearch start
Windows에서는 Elasticsearch가 설치된 디렉토리에서 bin/elasticsearch.bat 파일을 실행합니다.

로그 확인:
Elasticsearch 서버를 시작한 후 로그를 확인하여 어떤 오류도 없는지 확인하세요. 로그는 주로 {Elasticsearch 설치 디렉토리}/logs에 있습니다.

참고: Elasticsearch의 설정은 다양한 측면에서 수정될 수 있으며, 각 설정의 역할에 대해 이해하는 것이 중요합니다. 변경 사항이 Elasticsearch 클러스터에 영향을 미칠 수 있으므로 주의 깊게 수정해야 합니다. 또한, Elasticsearch의 버전에 따라 설정 항목이 다를 수 있으므로 해당 버전의 공식 문서를 참고하는 것이 좋습니다.